In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
# !pip install -q tensorflow-datasets
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                9600      
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 169,997
Trainable params: 169,997
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 5
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/5


185/782 [======>.......................] - ETA: 1:02:52 - loss: 0.6931 - accuracy: 0.437 - ETA: 1:21 - loss: 0.6930 - accuracy: 0.4609  - ETA: 1:22 - loss: 0.6930 - accuracy: 0.46 - ETA: 1:22 - loss: 0.6930 - accuracy: 0.46 - ETA: 1:20 - loss: 0.6930 - accuracy: 0.47 - ETA: 1:20 - loss: 0.6930 - accuracy: 0.47 - ETA: 1:20 - loss: 0.6930 - accuracy: 0.47 - ETA: 1:20 - loss: 0.6930 - accuracy: 0.48 - ETA: 1:20 - loss: 0.6930 - accuracy: 0.48 - ETA: 1:19 - loss: 0.6930 - accuracy: 0.48 - ETA: 1:19 - loss: 0.6930 - accuracy: 0.48 - ETA: 1:19 - loss: 0.6930 - accuracy: 0.48 - ETA: 1:19 - loss: 0.6930 - accuracy: 0.48 - ETA: 1:19 - loss: 0.6929 - accuracy: 0.49 - ETA: 1:19 - loss: 0.6929 - accuracy: 0.49 - ETA: 1:19 - loss: 0.6929 - accuracy: 0.49 - ETA: 1:19 - loss: 0.6929 - accuracy: 0.49 - ETA: 1:19 - loss: 0.6930 - accuracy: 0.49 - ETA: 1:18 - loss: 0.6930 - accuracy: 0.49 - ETA: 1:18 - loss: 0.6930 - accuracy: 0.49 - ETA: 1:18 - loss: 0.6931 - accuracy: 0.49 - ETA: 1:18 - loss: 0.6931 -

557/782 [====================>.........] - ETA: 44s - loss: 0.6890 - accuracy: 0.532 - ETA: 44s - loss: 0.6889 - accuracy: 0.532 - ETA: 43s - loss: 0.6888 - accuracy: 0.532 - ETA: 43s - loss: 0.6888 - accuracy: 0.532 - ETA: 43s - loss: 0.6887 - accuracy: 0.532 - ETA: 43s - loss: 0.6886 - accuracy: 0.533 - ETA: 43s - loss: 0.6886 - accuracy: 0.533 - ETA: 43s - loss: 0.6885 - accuracy: 0.533 - ETA: 43s - loss: 0.6884 - accuracy: 0.533 - ETA: 43s - loss: 0.6883 - accuracy: 0.533 - ETA: 43s - loss: 0.6883 - accuracy: 0.533 - ETA: 42s - loss: 0.6882 - accuracy: 0.533 - ETA: 42s - loss: 0.6881 - accuracy: 0.534 - ETA: 42s - loss: 0.6880 - accuracy: 0.534 - ETA: 42s - loss: 0.6879 - accuracy: 0.534 - ETA: 42s - loss: 0.6879 - accuracy: 0.534 - ETA: 42s - loss: 0.6878 - accuracy: 0.534 - ETA: 42s - loss: 0.6877 - accuracy: 0.534 - ETA: 42s - loss: 0.6876 - accuracy: 0.535 - ETA: 42s - loss: 0.6875 - accuracy: 0.535 - ETA: 41s - loss: 0.6875 - accuracy: 0.535 - ETA: 41s - loss: 0.6874 - accurac

745/782 [===========================>..] - ETA: 24s - loss: 0.6682 - accuracy: 0.562 - ETA: 24s - loss: 0.6680 - accuracy: 0.563 - ETA: 23s - loss: 0.6679 - accuracy: 0.563 - ETA: 23s - loss: 0.6678 - accuracy: 0.563 - ETA: 23s - loss: 0.6676 - accuracy: 0.563 - ETA: 23s - loss: 0.6675 - accuracy: 0.563 - ETA: 23s - loss: 0.6674 - accuracy: 0.563 - ETA: 23s - loss: 0.6672 - accuracy: 0.564 - ETA: 23s - loss: 0.6671 - accuracy: 0.564 - ETA: 23s - loss: 0.6670 - accuracy: 0.564 - ETA: 23s - loss: 0.6668 - accuracy: 0.564 - ETA: 22s - loss: 0.6667 - accuracy: 0.564 - ETA: 22s - loss: 0.6666 - accuracy: 0.564 - ETA: 22s - loss: 0.6664 - accuracy: 0.564 - ETA: 22s - loss: 0.6663 - accuracy: 0.565 - ETA: 22s - loss: 0.6662 - accuracy: 0.565 - ETA: 22s - loss: 0.6660 - accuracy: 0.565 - ETA: 22s - loss: 0.6659 - accuracy: 0.565 - ETA: 22s - loss: 0.6657 - accuracy: 0.565 - ETA: 22s - loss: 0.6656 - accuracy: 0.565 - ETA: 21s - loss: 0.6655 - accuracy: 0.566 - ETA: 21s - loss: 0.6653 - accurac

782/782 [==============================] - ETA: 3s - loss: 0.6424 - accuracy: 0.59 - ETA: 3s - loss: 0.6422 - accuracy: 0.59 - ETA: 3s - loss: 0.6421 - accuracy: 0.59 - ETA: 3s - loss: 0.6420 - accuracy: 0.59 - ETA: 3s - loss: 0.6418 - accuracy: 0.59 - ETA: 3s - loss: 0.6417 - accuracy: 0.59 - ETA: 3s - loss: 0.6416 - accuracy: 0.59 - ETA: 3s - loss: 0.6414 - accuracy: 0.59 - ETA: 2s - loss: 0.6413 - accuracy: 0.59 - ETA: 2s - loss: 0.6412 - accuracy: 0.59 - ETA: 2s - loss: 0.6410 - accuracy: 0.59 - ETA: 2s - loss: 0.6409 - accuracy: 0.59 - ETA: 2s - loss: 0.6408 - accuracy: 0.59 - ETA: 2s - loss: 0.6406 - accuracy: 0.59 - ETA: 2s - loss: 0.6405 - accuracy: 0.59 - ETA: 2s - loss: 0.6404 - accuracy: 0.59 - ETA: 2s - loss: 0.6402 - accuracy: 0.59 - ETA: 2s - loss: 0.6401 - accuracy: 0.59 - ETA: 1s - loss: 0.6400 - accuracy: 0.59 - ETA: 1s - loss: 0.6398 - accuracy: 0.59 - ETA: 1s - loss: 0.6397 - accuracy: 0.59 - ETA: 1s - loss: 0.6396 - accuracy: 0.59 - ETA: 1s - loss: 0.6394 - accuracy

186/782 [======>.......................] - ETA: 1:04 - loss: 0.2507 - accuracy: 0.93 - ETA: 1:04 - loss: 0.2979 - accuracy: 0.91 - ETA: 1:06 - loss: 0.3125 - accuracy: 0.90 - ETA: 1:06 - loss: 0.3121 - accuracy: 0.89 - ETA: 1:06 - loss: 0.3117 - accuracy: 0.89 - ETA: 1:06 - loss: 0.3101 - accuracy: 0.89 - ETA: 1:06 - loss: 0.3095 - accuracy: 0.88 - ETA: 1:06 - loss: 0.3067 - accuracy: 0.88 - ETA: 1:06 - loss: 0.3036 - accuracy: 0.88 - ETA: 1:06 - loss: 0.3016 - accuracy: 0.88 - ETA: 1:06 - loss: 0.3004 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2991 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2985 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2974 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2973 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2982 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2985 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2982 - accuracy: 0.88 - ETA: 1:06 - loss: 0.2981 - accuracy: 0.88 - ETA: 1:05 - loss: 0.2977 - accuracy: 0.88 - ETA: 1:05 - loss: 0.2978 - accuracy: 0.88 - ETA: 1:05 - loss: 0.2975 - accura

558/782 [====================>.........] - ETA: 35s - loss: 0.2871 - accuracy: 0.887 - ETA: 35s - loss: 0.2871 - accuracy: 0.887 - ETA: 35s - loss: 0.2871 - accuracy: 0.887 - ETA: 35s - loss: 0.2872 - accuracy: 0.887 - ETA: 35s - loss: 0.2872 - accuracy: 0.887 - ETA: 34s - loss: 0.2872 - accuracy: 0.887 - ETA: 34s - loss: 0.2872 - accuracy: 0.887 - ETA: 34s - loss: 0.2872 - accuracy: 0.887 - ETA: 34s - loss: 0.2873 - accuracy: 0.887 - ETA: 34s - loss: 0.2873 - accuracy: 0.887 - ETA: 34s - loss: 0.2873 - accuracy: 0.887 - ETA: 34s - loss: 0.2873 - accuracy: 0.887 - ETA: 34s - loss: 0.2873 - accuracy: 0.887 - ETA: 34s - loss: 0.2874 - accuracy: 0.887 - ETA: 34s - loss: 0.2874 - accuracy: 0.887 - ETA: 34s - loss: 0.2874 - accuracy: 0.887 - ETA: 33s - loss: 0.2874 - accuracy: 0.887 - ETA: 33s - loss: 0.2875 - accuracy: 0.887 - ETA: 33s - loss: 0.2875 - accuracy: 0.887 - ETA: 33s - loss: 0.2875 - accuracy: 0.887 - ETA: 33s - loss: 0.2875 - accuracy: 0.887 - ETA: 33s - loss: 0.2876 - accurac

638/782 [=======================>......] - ETA: 19s - loss: 0.2909 - accuracy: 0.884 - ETA: 19s - loss: 0.2909 - accuracy: 0.884 - ETA: 19s - loss: 0.2909 - accuracy: 0.884 - ETA: 19s - loss: 0.2909 - accuracy: 0.884 - ETA: 19s - loss: 0.2910 - accuracy: 0.884 - ETA: 18s - loss: 0.2910 - accuracy: 0.884 - ETA: 18s - loss: 0.2910 - accuracy: 0.884 - ETA: 18s - loss: 0.2910 - accuracy: 0.884 - ETA: 18s - loss: 0.2910 - accuracy: 0.884 - ETA: 18s - loss: 0.2910 - accuracy: 0.884 - ETA: 18s - loss: 0.2911 - accuracy: 0.884 - ETA: 18s - loss: 0.2911 - accuracy: 0.884 - ETA: 18s - loss: 0.2911 - accuracy: 0.884 - ETA: 18s - loss: 0.2911 - accuracy: 0.884 - ETA: 18s - loss: 0.2911 - accuracy: 0.884 - ETA: 18s - loss: 0.2911 - accuracy: 0.884 - ETA: 17s - loss: 0.2911 - accuracy: 0.884 - ETA: 17s - loss: 0.2912 - accuracy: 0.884 - ETA: 17s - loss: 0.2912 - accuracy: 0.884 - ETA: 17s - loss: 0.2912 - accuracy: 0.884 - ETA: 17s - loss: 0.2912 - accuracy: 0.884 - ETA: 17s - loss: 0.2912 - accurac

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [9]:
# Model Definition with LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 172,941
Trainable params: 172,941
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Model Definition with Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 116, 128)          10368     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 774       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,149
Trainable params: 171,149
Non-trainable params: 0
_________________________________________________________________
